In [ ]:
%pip install kaggle kagglehub plotly numpy pandas pillow matplotlib opencv-python ipywidgets ultralytics albumentations boto3

from IPython import display
display.clear_output()
!yolo checks

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("airbusgeo/airbus-aircrafts-sample-dataset")

print("Path to dataset files:", path)

In [ ]:
import numpy as np
import pandas as pd
import ast
import PIL
from pathlib import Path
import random
import cv2
import os
import boto3

import matplotlib
%matplotlib inline

In [ ]:
DATA_DIR = Path(path)
img_list = list(DATA_DIR.glob('images/*.jpg'))
pickone = random.choice(img_list)
display.Image(pickone)

In [ ]:
print(f"Found {len(img_list)} images files in {DATA_DIR}")

img = PIL.Image.open(pickone)
IMAGE_HEIGHT, IMAGE_WIDTH = img.size
num_channels = len(img.mode)
print("Image size: {}".format((IMAGE_HEIGHT, IMAGE_WIDTH)))
print("Num channels: {}".format(num_channels))

In [ ]:
df = pd.read_csv(DATA_DIR / 'annotations.csv')
# convert a string record into a valid python object
def f(x):
    return ast.literal_eval(x.rstrip('\r\n'))

df = pd.read_csv(DATA_DIR / "annotations.csv",
                converters={'geometry': f})
df.head(10)

In [ ]:
def getBounds(geometry):
    try:
        arr = np.array(geometry).T
        xmin = np.min(arr[0])
        ymin = np.min(arr[1])
        xmax = np.max(arr[0])
        ymax = np.max(arr[1])
        return (xmin, ymin, xmax, ymax)
    except:
        return np.nan

def getWidth(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(xmax - xmin)
    except:
        return np.nan

def getHeight(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(ymax - ymin)
    except:
        return np.nan

# Create bounds, width and height
df.loc[:,'bounds'] = df.loc[:,'geometry'].apply(getBounds)
df.loc[:,'width'] = df.loc[:,'bounds'].apply(getWidth)
df.loc[:,'height'] = df.loc[:,'bounds'].apply(getHeight)
df.head(10)

In [ ]:
# create a list of images used for validation
fold = 1
num_fold = 5
index = df['image_id'].unique()
val_indexes = index[len(index)*fold//num_fold:len(index)*(fold+1)//num_fold]
print(val_indexes)

In [ ]:
import tqdm.notebook

# Create 512x512 tiles with 64 pix overlap in /kaggle/working
TILE_WIDTH = 512
TILE_HEIGHT = 512
TILE_OVERLAP = 128
TRUNCATED_PERCENT = 0.3
_overwriteFiles = True

TILES_DIR = {'train': Path('kaggle/working/train/images/'),
             'val': Path('kaggle/working/val/images/')}
for _, folder in TILES_DIR.items():
    if not os.path.isdir(folder):
        os.makedirs(folder)

LABELS_DIR = {'train': Path('kaggle/working/train/labels/'),
              'val': Path('kaggle/working/val/labels/')}
for _, folder in LABELS_DIR.items():
    if not os.path.isdir(folder):
        os.makedirs(folder)

# Save one line in .txt file for each tag found inside the tile
def tag_is_inside_tile(bounds, x_start, y_start, width, height, truncated_percent):
    x_min, y_min, x_max, y_max = bounds
    x_min, y_min, x_max, y_max = x_min - x_start, y_min - y_start, x_max - x_start, y_max - y_start

    if (x_min > width) or (x_max < 0.0) or (y_min > height) or (y_max < 0.0):
        return None

    x_max_trunc = min(x_max, width)
    x_min_trunc = max(x_min, 0)
    if (x_max_trunc - x_min_trunc) / (x_max - x_min) < truncated_percent:
        return None

    y_max_trunc = min(y_max, height)
    y_min_trunc = max(y_min, 0)
    if (y_max_trunc - y_min_trunc) / (y_max - y_min) < truncated_percent:
        return None

    x_center = (x_min_trunc + x_max_trunc) / 2.0 / width
    y_center = (y_min_trunc + y_max_trunc) / 2.0 / height
    x_extend = (x_max_trunc - x_min_trunc) / width
    y_extend = (y_max_trunc - y_min_trunc) / height

    return (0, x_center, y_center, x_extend, y_extend)

for img_path in tqdm.notebook.tqdm(img_list):
    # Open image and related data
    pil_img = PIL.Image.open(img_path, mode='r')
    np_img = np.array(pil_img, dtype=np.uint8)

    # Get annotations for image
    img_labels = df[df["image_id"] == img_path.name]
    #print(img_labels)

    # Count number of sections to make
    X_TILES = (IMAGE_WIDTH + TILE_WIDTH + TILE_OVERLAP - 1) // TILE_WIDTH
    Y_TILES = (IMAGE_HEIGHT + TILE_HEIGHT + TILE_OVERLAP - 1) // TILE_HEIGHT

    # Cut each tile
    for x in range(X_TILES):
        for y in range(Y_TILES):

            x_end = min((x + 1) * TILE_WIDTH - TILE_OVERLAP * (x != 0), IMAGE_WIDTH)
            x_start = x_end - TILE_WIDTH
            y_end = min((y + 1) * TILE_HEIGHT - TILE_OVERLAP * (y != 0), IMAGE_HEIGHT)
            y_start = y_end - TILE_HEIGHT
            #print(x_start, y_start)

            folder = 'val' if img_path.name in val_indexes else 'train'
            save_tile_path = TILES_DIR[folder].joinpath(img_path.stem + "_" + str(x_start) + "_" + str(y_start) + ".jpg")
            save_label_path = LABELS_DIR[folder].joinpath(img_path.stem + "_" + str(x_start) + "_" + str(y_start) + ".txt")

            # Save if file doesn't exit
            if _overwriteFiles or not os.path.isfile(save_tile_path):
                cut_tile = np.zeros(shape=(TILE_WIDTH, TILE_HEIGHT, 3), dtype=np.uint8)
                cut_tile[0:TILE_HEIGHT, 0:TILE_WIDTH, :] = np_img[y_start:y_end, x_start:x_end, :]
                cut_tile_img = PIL.Image.fromarray(cut_tile)
                cut_tile_img.save(save_tile_path)

            found_tags = [
                tag_is_inside_tile(bounds, x_start, y_start, TILE_WIDTH, TILE_HEIGHT, TRUNCATED_PERCENT)
                for i, bounds in enumerate(img_labels['bounds'])]
            found_tags = [el for el in found_tags if el is not None]

            # save labels
            with open(save_label_path, 'w+') as f:
                for tags in found_tags:
                    f.write(' '.join(str(x) for x in tags) + '\n')

In [ ]:
CONFIG = """
# train and val datasets (image directory or *.txt file with image paths)
train: train/
val: val/

# number of classes
nc: 1

# class names
names: ['Aircraft']
"""

with open("kaggle/working/data.yaml", "w") as f:
    f.write(CONFIG)

In [ ]:
HOME = "kaggle/working/"
!yolo task=detect mode=train model=yolo11s.pt data={HOME}/data.yaml epochs=10 imgsz=512 augment=True auto_augment=True device=0

In [ ]:
train_path = 'runs/detect/train'

display.Image(filename=train_path + '/BoxF1_curve.png', width=600)

In [ ]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
pio.renderers.default = 'notebook'

df = pd.read_csv(train_path + "/results.csv")
fig = px.line(df, x='epoch', y='metrics/mAP50(B)', title='mAP50')
fig.show()

In [ ]:
display.Image(filename=train_path + '/val_batch0_pred.jpg', width=1000)

In [ ]:
!yolo task=detect mode=val model={train_path}/weights/best.pt data={HOME}/data.yaml device=0

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load your trained model
model_path = train_path + "/weights/best.pt"  # Update this path if needed
model = YOLO(model_path)

print(f"(Model loaded)")

In [ ]:
# Path to your test images folder
test_images_path = path + "/extras"  # Change this to your test images folder

# Get all images in the test folder
test_images = [str(p) for p in Path(test_images_path).glob("*.jpg")]
print(f"Found {len(test_images)} test images")

In [ ]:
# Tiled Inference for Full-Size Images
# This notebook implements sliding window inference to match the training tile size

# Configuration - MUST match training parameters
TILE_WIDTH = 512
TILE_HEIGHT = 512
TILE_OVERLAP = 128
CONF_THRESHOLD = 0.15
NMS_THRESHOLD = 0.8  # Non-maximum suppression for overlapping detections

def sliding_window_inference(image_path, model, tile_width=512, tile_height=512, overlap=64, conf_threshold=0.25):
    """
    Perform inference on full-size image using sliding window approach

    Args:
        image_path: Path to the full-size image
        model: Trained YOLO model
        tile_width: Width of each tile (should match training)
        tile_height: Height of each tile (should match training)
        overlap: Overlap between tiles (should match training)
        conf_threshold: Confidence threshold for detections

    Returns:
        List of detections with format: [x1, y1, x2, y2, confidence, class_id]
    """
    # Load the full image
    image = Image.open(image_path)
    img_width, img_height = image.size
    image_np = np.array(image)

    print(f"Processing image: {os.path.basename(image_path)}")
    print(f"Image size: {img_width}x{img_height}")

    all_detections = []

    # Calculate number of tiles needed
    x_tiles = (img_width + tile_width - overlap - 1) // (tile_width - overlap)
    y_tiles = (img_height + tile_height - overlap - 1) // (tile_height - overlap)

    print(f"Creating {x_tiles}x{y_tiles} = {x_tiles * y_tiles} tiles")

    for y in range(y_tiles):
        for x in range(x_tiles):
            # Calculate tile boundaries
            x_start = x * (tile_width - overlap)
            y_start = y * (tile_height - overlap)
            x_end = min(x_start + tile_width, img_width)
            y_end = min(y_start + tile_height, img_height)

            # Extract tile
            tile = image_np[y_start:y_end, x_start:x_end]

            # Pad tile if necessary to maintain consistent input size
            if tile.shape[0] != tile_height or tile.shape[1] != tile_width:
                padded_tile = np.zeros((tile_height, tile_width, 3), dtype=np.uint8)
                padded_tile[:tile.shape[0], :tile.shape[1]] = tile
                tile = padded_tile

            # Convert to PIL Image for YOLO
            tile_pil = Image.fromarray(tile)

            # Run inference on tile
            results = model(tile_pil, conf=conf_threshold, verbose=False)

            # Process detections from this tile
            if len(results[0].boxes) > 0:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                scores = results[0].boxes.conf.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()

                # Convert tile coordinates to full image coordinates
                for box, score, cls in zip(boxes, scores, classes):
                    x1, y1, x2, y2 = box

                    # Adjust coordinates to full image space
                    global_x1 = x1 + x_start
                    global_y1 = y1 + y_start
                    global_x2 = x2 + x_start
                    global_y2 = y2 + y_start

                    # Option B: Remove margin completely for center tiles
                    if x > 0 and x < x_tiles-1 and y > 0 and y < y_tiles-1:
                        # Center tiles - no margin needed
                        all_detections.append([global_x1, global_y1, global_x2, global_y2, score, int(cls)])
                    else:
                        # Edge tiles - small margin to avoid image boundary issues
                        margin = 10
                        if (x1 > margin and y1 > margin and
                            x2 < tile_width - margin and y2 < tile_height - margin):
                            all_detections.append([global_x1, global_y1, global_x2, global_y2, score, int(cls)])


    print(f"Found {len(all_detections)} raw detections before NMS")

    # Apply Non-Maximum Suppression to remove duplicate detections
    if len(all_detections) > 0:
        detections_array = np.array(all_detections)

        # Extract boxes and scores for NMS
        boxes = detections_array[:, :4]
        scores = detections_array[:, 4]

        # Apply NMS using OpenCV
        indices = cv2.dnn.NMSBoxes(
            boxes.tolist(),
            scores.tolist(),
            conf_threshold,
            NMS_THRESHOLD
        )

        if len(indices) > 0:
            final_detections = detections_array[indices.flatten()]
            print(f"Final detections after NMS: {len(final_detections)}")
            return final_detections.tolist()

    print("No detections found")
    return []

def visualize_tiled_detections(image_path, detections, save_path=None):
    """
    Visualize detections on the full-size image with confidence-based color coding
    """
    # Load the image
    image = Image.open(image_path)
    img_np = np.array(image)

    # Create figure
    fig, ax = plt.subplots(1, figsize=(15, 15))
    ax.imshow(img_np)

    # Count detections by confidence level
    high_conf = sum(1 for det in detections if det[4] >= 0.7)
    med_conf = sum(1 for det in detections if 0.5 <= det[4] < 0.7)
    low_conf = sum(1 for det in detections if 0.25 <= det[4] < 0.5)

    # Draw detections with confidence-based colors
    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        width = x2 - x1
        height = y2 - y1

        # Choose color and label background based on confidence level
        if confidence >= 0.7:
            edge_color = 'red'
            label_color = 'red'
            conf_level = 'HIGH'
        elif confidence >= 0.5:
            edge_color = 'orange'
            label_color = 'orange'
            conf_level = 'MED'
        else:  # confidence >= 0.25
            edge_color = 'yellow'
            label_color = 'gold'
            conf_level = 'LOW'

        # Create rectangle with confidence-based color
        rect = patches.Rectangle((x1, y1), width, height,
                               linewidth=2, edgecolor=edge_color, facecolor='none')
        ax.add_patch(rect)

        # Add label with confidence level indicator
        ax.text(x1, y1-10, f'Aircraft: {confidence:.2f} ({conf_level})',
               bbox=dict(facecolor=label_color, alpha=0.7),
               color='white', fontsize=10, weight='bold')

    # Enhanced title with confidence breakdown
    title = f'Tiled Inference Results: {os.path.basename(image_path)}\n'
    title += f'Total: {len(detections)} detections '
    title += f'(High≥0.7: {high_conf}, Med≥0.5: {med_conf}, Low≥0.25: {low_conf})'

    ax.set_title(title, fontsize=12, pad=20)
    ax.axis('off')

    # Add legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='red', lw=2, label='High Confidence (≥0.7)'),
        Line2D([0], [0], color='orange', lw=2, label='Medium Confidence (0.5-0.7)'),
        Line2D([0], [0], color='yellow', lw=2, label='Low Confidence (0.25-0.5)')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        print(f"Saved to: {save_path}")

    plt.tight_layout()
    plt.show()
    plt.close()



# Create output directory
output_dir = "tiled_inference_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each test image with tiled inference
for i, img_path in enumerate(test_images):
    print(f"\n{'='*50}")
    print(f"Processing image {i+1}/{len(test_images)}: {os.path.basename(img_path)}")
    print(f"{'='*50}")

    # Run sliding window inference
    detections = sliding_window_inference(
        img_path,
        model,
        tile_width=TILE_WIDTH,
        tile_height=TILE_HEIGHT,
        overlap=TILE_OVERLAP,
        conf_threshold=CONF_THRESHOLD
    )

    # Visualize results
    save_path = os.path.join(output_dir, f"tiled_{os.path.basename(img_path)}")
    visualize_tiled_detections(img_path, detections, save_path)

## Comparison: Direct vs Tiled Inference

# Compare direct inference vs tiled inference for one image
test_image = test_images[0]  # Change this to test different images

print("=== DIRECT INFERENCE (Original Method) ===")
# Direct inference on full image
results_direct = model(test_image, conf=CONF_THRESHOLD)
direct_detections = len(results_direct[0].boxes) if len(results_direct[0].boxes) > 0 else 0
print(f"Direct inference detections: {direct_detections}")

print("\n=== TILED INFERENCE (New Method) ===")
# Tiled inference
tiled_detections = sliding_window_inference(test_image, model, conf_threshold=CONF_THRESHOLD)
print(f"Tiled inference detections: {len(tiled_detections)}")

print(f"\n=== SUMMARY ===")
print(f"Improvement: {len(tiled_detections) - direct_detections} additional detections")

In [ ]:
# Export model to ONNX format
model.export(format='onnx', imgsz=512)

# Get S3 credentials from environment variables (pre-configured in OpenShift AI)
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_S3_ENDPOINT = os.getenv('AWS_S3_ENDPOINT')
AWS_S3_BUCKET = os.getenv('AWS_S3_BUCKET')
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION', 'us-east-1')

print("S3 Configuration:")
print(f"Raw Endpoint: {AWS_S3_ENDPOINT}")
print(f"Bucket: {AWS_S3_BUCKET}")
print(f"Region: {AWS_DEFAULT_REGION}")

# Fix the endpoint URL - add https:// if missing
if AWS_S3_ENDPOINT and not AWS_S3_ENDPOINT.startswith(('http://', 'https://')):
    # For internal OpenShift storage, use https://
    endpoint_url = f"https://{AWS_S3_ENDPOINT}"
else:
    endpoint_url = AWS_S3_ENDPOINT

print(f"Fixed Endpoint: {endpoint_url}")

# Set up S3 client using environment variables
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    endpoint_url=endpoint_url,
    region_name=AWS_DEFAULT_REGION,
)

# OpenVINO Model Server configuration
bucket_name = AWS_S3_BUCKET
model_name = 'airplane-detection'
model_version = '1'

# Correct OpenVINO Model Server folder structure: models/{model_name}/{version}/model.onnx
model_key = f'models/{model_name}/{model_version}/model.onnx'
onnx_path = train_path + "/weights/best.onnx"

print(f"Uploading model to OpenVINO Model Server structure...")
print(f"Bucket: {bucket_name}")
print(f"Model path: {model_key}")

try:
    # Upload the ONNX model file
    s3_client.upload_file(onnx_path, bucket_name, model_key)
    print(f"✅ Model uploaded to s3://{bucket_name}/{model_key}")

    # Create and upload model configuration file (optional but recommended)
    import json
    model_config = {
        "model_config_list": [
            {
                "config": {
                    "name": model_name,
                    "base_path": f"/models/{model_name}",
                    "model_platform": "onnxruntime_onnx",
                    "model_version_policy": {"all": {}},
                    "instance_group": [
                        {
                            "name": "aircraft_detection",
                            "kind": "KIND_CPU",
                            "count": 1
                        }
                    ]
                }
            }
        ]
    }

    # Save config locally first
    config_path = "model_config.json"
    with open(config_path, 'w') as f:
        json.dump(model_config, f, indent=2)

    # Upload config file
    config_key = f'models/{model_name}/{model_version}/config.json'
    s3_client.upload_file(config_path, bucket_name, config_key)
    print(f"✅ Config uploaded to s3://{bucket_name}/{config_key}")

    # Create model server configuration for easier deployment
    server_config = {
        "model_config_list": [
            {
                "config": {
                    "name": model_name,
                    "base_path": f"s3://{bucket_name}/models/{model_name}",
                    "model_platform": "onnxruntime_onnx",
                    "model_version_policy": {"all": {}}
                }
            }
        ]
    }

    server_config_path = "server_config.json"
    with open(server_config_path, 'w') as f:
        json.dump(server_config, f, indent=2)

    print(f"\n📋 Model Server Configuration created: {server_config_path}")
    print(f"Use this file to start OpenVINO Model Server with your model")

    # Verify the upload structure
    print(f"\n🔍 Verifying S3 structure...")
    try:
        # List objects in the model directory
        response = s3_client.list_objects_v2(
            Bucket=bucket_name,
            Prefix=f'models/{model_name}/{model_version}/',
            Delimiter='/'
        )

        if 'Contents' in response:
            print("✅ Files uploaded successfully:")
            for obj in response['Contents']:
                print(f"   📄 {obj['Key']} ({obj['Size']:,} bytes)")
        else:
            print("⚠️  No files found - upload may have failed")

    except Exception as verify_error:
        print(f"⚠️  Could not verify upload: {verify_error}")

except Exception as e:
    print(f"❌ Upload failed: {e}")
    print(f"\nTroubleshooting:")
    print(f"1. Check your AWS credentials and permissions")
    print(f"2. Verify the bucket '{bucket_name}' exists and is accessible")
    print(f"3. Ensure your S3 endpoint URL is correct")

# Clean up temporary files
import os
try:
    if os.path.exists("model_config.json"):
        os.remove("model_config.json")
    if os.path.exists("server_config.json"):
        # Keep this file for deployment reference
        print(f"\n📁 Server configuration saved as: server_config.json")
        print(f"   You can use this file to deploy your model server")
except:
    pass
